In [62]:

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

from openai import OpenAI
import pandas as pd

In [63]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [64]:
# load document
loader = PyPDFLoader("../../example_paper1.pdf")
documents = loader.load()
print(len(documents))

33


In [65]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=0)
text = text_splitter.split_documents(documents)
print(len(text))

43


In [61]:
len(documents[0].page_content)


3527

In [66]:
import regex as re
class EvalData:

    def __init__(self, question_answer_pair):
        self.question_answer_pair = question_answer_pair
        self.questions = None
    
    
    def call_openai_chat_completion(text_chunk):

        client = OpenAI()

        # text_chunk = text[0].page_content
        content = text_chunk
        n = 2

        prompt = f"""Please generate {n} questions asking for the key information in the given paragraph. 
            Also answer the questions using the information in the given paragraph. 
            Please specific questions rather than general questeions, like 'what is the key information in the given paragraph?'.
            Questions should end with '?'. 
            Please generate answers using as much information as possible.
            If you are unable to answer it, please generate the answer as 'sorry I don't know.'
            The output should be formatted. Every question-answer pair should be separated by '\n\n'. Questions should start with "Question:" and answers should start with "Answer:". 
            Each question and answer should be saparated by '\n'. 

            paragraph: {content}
            """
 

        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model = "gpt-3.5-turbo",   
            # tools = [{"type": "function", "function": submit_function}]
            # response_format={"type":"json_object"}
        )

        generated = response.choices[0].message.content
        qa_list = [qa_pair.split('\n') for qa_pair in generated.split('\n\n')]
        reformated_qa_list = [dict(question=re.sub(r"Question: ", "", qa_pair[0]), answer=re.sub(r"Answer: ", "", qa_pair[1])) for qa_pair in qa_list]       
        return reformated_qa_list
        # return response.choices[0].message.content

    @classmethod
    def from_llm(cls, input):

        return cls(cls.call_openai_chat_completion(input))

    def add_data(self, questions, question_with_anwers = None):

        self.questions = questions
        return self
     
    def get_all_questions(self):
        
        test_df = pd.DataFrame([], columns=["Question", "Answer", "anwerable", "attack_type"])

        if self.questions is not None:
            add_questions = [dict(question=q, answer="none") for q in self.questions]
            qa_list = self.question_answer_pair + add_questions
            test_data = pd.DataFrame(qa_list)
            test_df = test_df.assign(Question=test_data["question"], Answer=test_data["answer"])
        else:
            test_data = pd.DataFrame(self.question_answer_pair)    
            test_df = test_df.assign(Question=test_data["question"], Answer=test_data["answer"])
        return test_df
               

In [54]:
ql = ["what is the key conclusion of this text?"]

In [70]:
text_chunk = text[1].page_content
eval_data = EvalData.from_llm(text_chunk)

In [71]:
eval_data.get_all_questions()

,Question,Answer,anwerable,attack_type
0,What are some other sources of unreplicability...,The paragraph mentions unequal hyperparameters...,NaN,NaN
1,What does the tabular data benchmark with a pr...,The tabular data benchmark with a precise meth...,NaN,NaN


In [72]:
eval_data.add_data(ql)

In [73]:
t = eval_data.question_answer_pair
t

[{'question': 'What are some other sources of unreplicability across machine learning mentioned in the paragraph?',
  'answer': 'The paragraph mentions unequal hyperparameters tuning efforts [Lipton and Steinhardt, 2019] and failure to account for statistical uncertainty in benchmarks [Bouthillier et al., 2021] as other sources of unreplicability across machine learning.'},
 {'question': 'What does the tabular data benchmark with a precise methodology enable the authors to do?',
  'answer': 'The tabular data benchmark with a precise methodology enables the authors to evaluate recent deep learning models that have not yet been independently evaluated.'}]